In [1]:
import Main
import Parser
from Analyser import *

In [2]:
modelName = 'dereffye'

In [3]:
# Some PyABC magic to get our set of parameters
from pyabc import (ABCSMC, Distribution, RV, MultivariateNormalTransition)
import scipy as sp
import scipy.stats as st
import tempfile
import os
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def model(params):
    return Main.generate_and_analyse(modelName, params)

In [5]:
t1 = Main.generate_and_analyse(modelName, {})
t1

{'parentAngle_mean': 17.699175030385526,
 'parentAngle_variance': 21.63794032878498,
 'length_mean': 1.4055027827336024,
 'length_variance': 0.9057108250017012,
 'verticalOrientation_mean': 52.34535954385513,
 'verticalOrientation_variance': 28.606027771964857,
 'horizontalOrientation_mean': 170.775781851045,
 'horizontalOrientation_variance': 96.99397281210354,
 'q1_xyz': (4.8931391318170725, 27.44963686772176, 10.910246326216436),
 'q2_xyz': (51.634732127126014, 25.029195913598265, 50.331208023955185),
 'q3_xyz': (51.07175846300456, 27.822816617962992, 45.22054969894489),
 'xyz': (52.72120641920593, 79.74730372746824, 53.003617918787754),
 'trunk_length': 10.0,
 'totalBiomass': 397.23712082645477}

In [6]:
metrics = {  'slash':(10, 100),
             'slashr':(10, 100),
             'plus':(0, 180),
             'plusr':(0, 180),
             'decay':(0, 1),
             'd': (0, 1),
             'h':(1, 20),
             'w':(0.1, 2),
             'thickness':(0, 4),
             'depth':(1,7)
          }

In [7]:
# prior = Distribution(section_1_slash     =RV("uniform", 10, 100),
#                      section_1_slashr    =RV("uniform", 10, 100),
#                      section_1_plus      =RV("uniform", 0, 180),
#                      section_1_plusr      =RV("uniform", 0, 180),
#                      section_1_decay     =RV("uniform", 0, 1),
#                      section_1_d         =RV("uniform", 0,1),
#                      section_1_h         =RV("uniform", 1, 20),
#                      section_1_w         =RV("uniform", 0.1, 2),
#                      general_thickness   =RV("uniform", 0, 5)
#                     )

In [8]:
slash = Distribution(section_1_slash     =RV("uniform", 10, 100))
slashr = Distribution(section_1_slashr    =RV("uniform", 10, 100))
plus = Distribution(section_1_plus      =RV("uniform", 0, 180))
plusr = Distribution(section_1_plusr      =RV("uniform", 0, 180))
decay = Distribution(section_1_decay     =RV("uniform", 0, 1))
d = Distribution(section_1_d     =RV("uniform", 0, 1))
h = Distribution(section_1_h         =RV("uniform", 1, 20))
w = Distribution(section_1_w         =RV("uniform", 0.1, 2))
thickness = Distribution(general_thickness   =RV("uniform", 0, 5))
depth = Distribution(general_depth           =RV("uniform",1,7))

In [9]:
db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "test.db"))

In [10]:
def gen_av_target(n):
    target = model({})
    for i in range(n):
        print(i)
        new_metrics = model({})
        for (k,v) in new_metrics.items():
            if k[-3:] == 'xyz':
                x,y,z = target[k]
                x += new_metrics[k][0]
                y += new_metrics[k][1]
                z += new_metrics[k][2]
                target[k] = (x,y,z)
            else:
                target[k] += new_metrics[k]

    for (k,v) in target.items():
        if k[-3:] == 'xyz':
            x,y,z = target[k]
            target[k] = (x/n, y/n, z/n)
        else:
            target[k] /= n
            
    return target

In [11]:
target = gen_av_target(20)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [12]:
target

{'parentAngle_mean': 18.35232150118623,
 'parentAngle_variance': 22.42427236586422,
 'length_mean': 1.4757779218702827,
 'length_variance': 0.9509963662517865,
 'verticalOrientation_mean': 58.58261736151267,
 'verticalOrientation_variance': 31.437050536908753,
 'horizontalOrientation_mean': 177.77525344071296,
 'horizontalOrientation_variance': 108.22474314321762,
 'q1_xyz': (21.12223563442419, 28.45541662019491, 29.228414091148778),
 'q2_xyz': (52.81401117415142, 26.473555815752178, 58.70481678704196),
 'q3_xyz': (47.08066094974712, 29.179083350224158, 47.9304868688608),
 'xyz': (55.673950447377294, 83.73466891384166, 63.07518124075746),
 'trunk_length': 10.5,
 'totalBiomass': 417.0989768677776}

In [13]:
mp = 7
me = 0.5

In [14]:
abc = ABCSMC(model, slash, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

INFO:History:Start <ABCSMC(id=54, start_time=2019-04-17 11:33:55.796206, end_time=None)>
INFO:Epsilon:initial epsilon is 6.246676412890714
INFO:ABC:t:0 eps:6.246676412890714
INFO:ABC:t:1 eps:4.270630871017434
Process Process-21:
Process Process-19:
Process Process-18:
Process Process-17:
Process Process-24:
Process Process-23:
Process Process-20:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/t/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/t/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/t/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/t/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/t/anaco

  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/smc.py", line 644, in _evaluate_proposal
    x_0)
  File "/home/t/Documents/diss/CompiledApproach/treeMod8000/App/Models/Turtle.py", line 27, in View
    horizontalAngle = rotations.angleBetween([1,0,0], self.heading[:,0])
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/smc.py", line 553, in simulate_one
    transitions)
  File "/home/t/Documents/diss/CompiledApproach/treeMod8000/App/Models/Main.py", line 110, in generate_and_analyse
    tree_data = generate_model(filename, overrideParams, return_analysis=True)
  File "/home/t/Documents/diss/CompiledApproach/treeMod8000/App/Models/Deriver.py", line 16, in derive
    new_tree += element.Grow()
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/model.py", line 116, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/smc.py", line 644, in _evaluate_proposal
    x_0)
  File "/home/t/anaconda3/lib/p

  File "/home/t/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py", line 2428, in array_equal
    return bool(asarray(a1 == a2).all())
KeyboardInterrupt
KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/t/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-fe89220206f1>", line 3, in <module>
    history = abc.run(minimum_epsilon=me, max_nr_populations=mp)
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/smc.py", line 794, in run
    self.population_strategy.nr_particles, simulate_one)
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 111, in sample_until_n_accepted
    val = get_if_worker_healthy(processes, queue)
  File "/home/t/anaconda3/lib/python3.7/site-packages/pyabc/sampler/multicorebase.py", line 53, in get_if_worker_healthy
    item = queue.get(True, 5)
  File "/home/t/anaconda3/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/t/anaconda3/lib/python3.7/site-packages/pandas/core

KeyboardInterrupt: 

In [ ]:
from pyabc.visualization import plot_kde_1d

In [ ]:
# Visualise results
k = 'slash'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_slash', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, slashr, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'slashr'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0]-1, xmax=1.5*v[1],
                x='section_1_slashr', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, plus, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'plus'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_plus', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, plusr, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'plusr'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_plusr', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, decay, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'decay'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_decay', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, d, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'd'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_d', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, h, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'h'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='section_1_h', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, w, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'w'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=0, xmax=0.6,
                x='section_1_w', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, thickness, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'thickness'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='general_thickness', ax=ax,
                label="PDF t={}".format(t))
ax.legend();

In [ ]:
abc = ABCSMC(model, depth, distance)
abc.new(db_path, target)
history = abc.run(minimum_epsilon=me, max_nr_populations=mp)

In [ ]:
# Visualise results
k = 'depth'
v = metrics[k]
fig, ax = plt.subplots()
for t in range(history.max_t, history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    plot_kde_1d(df, w,
                xmin=v[0], xmax=1.5*v[1],
                x='general_depth', ax=ax,
                label="PDF t={}".format(t))
ax.legend();